# Shamir Secret Sharing (SS)

This is demo for our implementation of Shamir's secret sharing. We choose the Field $\mathbb{Z}_p$ where $p > 2^\lambda$ is a prime ($\lambda$ is the key size)

In [1]:
from ftsa.protocols.buildingblocks.ShamirSS import SSS
import random

threshold = 70
nclients  = 100
SS = SSS(1024)

number1 = random.randint(0,2**1024-1)
number2 = random.randint(0,2**1024-1)

allshares1 = SS.share(threshold, nclients, number1)
allshares2 = SS.share(threshold, nclients, number2)

random.seed(1)
shares1 = random.sample(allshares1, threshold)
random.seed(1)
shares2 = random.sample(allshares2, threshold)

shares = [x+y for x,y in zip(shares1, shares2)]

lagcoef = SS.lagrange(shares1)
reconstucted1 =  SS.recon(shares1, lagcoef)
reconstucted2 =  SS.recon(shares2, lagcoef)
reconstucted =  SS.recon(shares, lagcoef)

# check if the result is correct
print("Verify1: ", number1 == reconstucted1)
print("Verify2: ", number2 == reconstucted2)
print("Verify Sum: ", (number2 + number1) == reconstucted)


Verify1:  True
Verify2:  True
Verify Sum:  True


# Shamir Secret Sharing ove the Integers (ISS)

This is demo for our implementation of Shamir's secret sharing over the integers. We choose the security parameter sigma to $\sigma = 128$ bits

In [2]:
from ftsa.protocols.buildingblocks.IntegerSS import ISSS
import random
from math import factorial

sigma = 128
threshold = 70
nclients  = 100
delta = factorial(nclients)

SS = ISSS(1024, sigma)

number1 = random.randint(-2**1024+1,2**1024-1)
number2 = random.randint(-2**1024+1,2**1024-1)

allshares1 = SS.Share(number1,threshold, range(1,nclients+1))
allshares2 = SS.Share(number2,threshold, range(1,nclients+1))

random.seed(1)
shares1 = random.sample(allshares1, threshold)
random.seed(1)
shares2 = random.sample(allshares2, threshold)

shares = [x+y for x,y in zip(shares1, shares2)]

reconstucted1 =  SS.Recon(shares1, threshold, delta)
reconstucted2 =  SS.Recon(shares2, threshold, delta)
reconstucted =  SS.Recon(shares, threshold, delta)

# check if the result is correct
print("Verify1: ", number1 == reconstucted1)
print("Verify2: ", number2 == reconstucted2)
print("Verify Sum: ", (number2 + number1) == reconstucted)


Verify1:  True
Verify2:  True
Verify Sum:  True


# Deffie-Hellman Key Aggreement (KA)
It uses ECDH p256 curve. It then computes with SHA256 over the ECDH agreed key to generate a secret key for AES (256 bit)

In [3]:
from ftsa.protocols.buildingblocks.KeyAggreement import KAS

sharedkeysize = 1024

KA1 = KAS()
KA2 = KAS()

KA1.generate()
KA2.generate()

# check if the shared secrets match
print("Verify:", KA1.agree(KA2.pk, sharedkeysize) == KA2.agree(KA1.pk, sharedkeysize))

# print shared key
sharedkey = KA1.agree(KA2.pk, sharedkeysize)
print("key: " + str(sharedkey))

Verify: True
key: 163829413318910999754260423462676387308988117954797857438116280667141082990718384063456897117570711567278346198832493566731595501681391482873845226558874686279213996496802201619337650133595918904708436909176817931645078875087441686054129311603364945474037450254803001155241640425270441360056702443712224128970


In [4]:
# test serializing secret keys
sk1_bytes = KA1.get_sk_bytes()
KA = KAS()
KA.generate_from_bytes(sk1_bytes)
print("Verify:", KA.sk == KA1.sk and  KA.pk == KA1.pk)

Verify: True


# AES-GCM Authenticated Encryption (AE)

In [5]:
import ftsa.protocols.buildingblocks.AESGCM128 as aes

message=b"TOP SECRET"

key = aes.EncryptionKey(sharedkey)
e = key.encrypt(message)
m = key.decrypt(e)
print("Verify:", message == m)

Verify: True


# Vector Encoding Scheme (VE)
This is a vector to vector encoding. It batches several elements of the vectors into a single integer. The batch size depends on the size of that integer (i.e. this means the plaintext size of an encryption scheme). It also depends on the number of safety bits need to be preserved for each vector element (this is to support addition operation without reaching an overflow)

In [6]:
from ftsa.protocols.buildingblocks.VectorEncoding import VES

keysize = 1024
nclients = 100
valuesize = 16
vectorsize = 100

VE = VES(keysize, nclients, valuesize, vectorsize)
vector = list(range(1,vectorsize+1))

# encode the vector 
E = VE.encode(V=vector)
print("encoded decreased the vector size from {} to {}".format(len(vector),len(E)))
# decode the vector 
V = VE.decode(E=E)
print("Verify:", V == vector)

encoded decreased the vector size from 100 to 3
Verify: True


# Pseudo Randomg Generator (PRG)
## We use AES-CTR to generate a pseudo random vector from a seed

The demo first generates a random seed. The seed is used as the AES key to encrypt a '0x00'-repeated message of large size. The encryption result is transformed into a vector of integers. The output vector size is controlled by the size of the input message.

In [7]:
from ftsa.protocols.buildingblocks.PRG import PRG
import random

securitybits = 128 #bits
elementsize = 16 #bits
vectorsize = 2**20 #elements

# setup the PRG 
prg = PRG(vectorsize, elementsize)

In [8]:
# get a random seed of 16 bytes long
b = random.SystemRandom().getrandbits(securitybits).to_bytes(securitybits // 8,"big")

# generate a random vector from that seed 
B = prg.eval(b)

print("Generated a pseudo random vector of length {} ({} bits elements) from {} bits random value".format(len(B), elementsize, securitybits))
B

Generated a pseudo random vector of length 1048576 (16 bits elements) from 128 bits random value


[17688,
 60242,
 16871,
 57394,
 567,
 2312,
 43437,
 65240,
 46882,
 30694,
 42934,
 4075,
 61553,
 9631,
 42731,
 64954,
 14006,
 2993,
 32536,
 19097,
 46821,
 53308,
 3528,
 21586,
 9295,
 11826,
 15344,
 47254,
 13038,
 8918,
 15580,
 31911,
 33595,
 17951,
 2486,
 48206,
 39322,
 48346,
 28938,
 27942,
 40157,
 64920,
 38585,
 2124,
 1532,
 38796,
 63473,
 20614,
 11581,
 12241,
 23501,
 13855,
 11268,
 2343,
 26362,
 57778,
 30325,
 24500,
 24817,
 22819,
 48586,
 1770,
 10982,
 43843,
 16625,
 59410,
 18425,
 163,
 21454,
 43919,
 30516,
 4537,
 31223,
 40759,
 25693,
 59394,
 827,
 62951,
 19505,
 7067,
 32765,
 45023,
 64776,
 55695,
 32669,
 1597,
 30319,
 43997,
 35918,
 64775,
 43929,
 65484,
 16181,
 10536,
 52101,
 30644,
 44209,
 17785,
 7929,
 47450,
 57952,
 65107,
 45358,
 35568,
 4039,
 46036,
 14904,
 38054,
 42270,
 53142,
 154,
 59189,
 5718,
 61898,
 61306,
 39533,
 20976,
 34160,
 64296,
 62233,
 5380,
 12593,
 53204,
 16560,
 1676,
 42602,
 4770,
 13112,
 6499